# Analyzing RCT data with Precision Adjustment

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. 
  

In [35]:
## loading the data
Penn <- as.data.frame(read.table("../data/penn_jae.dat", header=T ))
n <- dim(Penn)[1]
p_1 <- dim(Penn)[2]
Penn<- subset(Penn, tg==2| tg==0)
attach(Penn)

The following objects are masked from Penn (pos = 4):

    abdt, agegt54, agelt35, black, dep, durable, female, hispanic,
    husd, inuidur1, inuidur2, lusd, muld, nondurable, othrace, q1, q2,
    q3, q4, q5, q6, recall, tg


The following objects are masked from Penn (pos = 8):

    abdt, agegt54, agelt35, black, dep, durable, female, hispanic,
    husd, inuidur1, inuidur2, lusd, muld, nondurable, othrace, q1, q2,
    q3, q4, q5, q6, recall, tg




In [36]:
T2<- (tg==2)
summary(T2)

   Mode   FALSE    TRUE 
logical    3354    2428 

In [37]:
head(Penn)

abdt  tg inuidur1 inuidur2 female black hispanic othrace dep q1 ... q5 q6
1  10824 0  18       18       0      0     0        0       2   0  ... 1  0 
2  10635 2   7        3       0      0     0        0       0   0  ... 0  0 
4  10824 0   1        1       0      0     0        0       0   0  ... 1  0 
5  10747 0  27       27       0      0     0        0       0   0  ... 0  0 
10 10754 2  20       20       1      0     0        0       0   0  ... 0  0 
13 10831 0  27       27       0      0     0        0       1   0  ... 1  0 
   recall agelt35 agegt54 durable nondurable lusd husd muld
1  0      0       0       0       0          0    1    0   
2  0      1       0       0       0          1    0    0   
4  0      0       0       0       0          1    0    0   
5  0      0       0       0       0          1    0    0   
10 0      0       0       0       0          0    0    1   
13 0      0       1       1       0          1    0    0

In [38]:
#summarize variables 
summary(Penn)

      abdt             tg            inuidur1        inuidur2    
 Min.   :10404   Min.   :0.0000   Min.   : 1.00   Min.   : 0.00  
 1st Qu.:10600   1st Qu.:0.0000   1st Qu.: 3.00   1st Qu.: 2.00  
 Median :10698   Median :0.0000   Median :11.00   Median :10.00  
 Mean   :10696   Mean   :0.8398   Mean   :13.03   Mean   :12.25  
 3rd Qu.:10796   3rd Qu.:2.0000   3rd Qu.:25.00   3rd Qu.:23.00  
 Max.   :10880   Max.   :2.0000   Max.   :52.00   Max.   :52.00  
     female           black           hispanic          othrace        
 Min.   :0.0000   Min.   :0.0000   Min.   :0.00000   Min.   :0.000000  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.00000   1st Qu.:0.000000  
 Median :0.0000   Median :0.0000   Median :0.00000   Median :0.000000  
 Mean   :0.4007   Mean   :0.1188   Mean   :0.03632   Mean   :0.005189  
 3rd Qu.:1.0000   3rd Qu.:0.0000   3rd Qu.:0.00000   3rd Qu.:0.000000  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.00000   Max.   :1.000000  
      dep               q1        

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

and carry out robust inference using the *estimatr* R packages. 

# Carry out covariate balance check

This is done using "lm_robust" command which unlike "lm" in the base command automatically does the correct Eicher-Huber-White standard errors, instead othe classical non-robus formula based on the homoscdedasticity command.

In [39]:
#install.packages("lmtest")
#install.packages("sandwich")

library(lmtest)
library(sandwich)

In [40]:
m <- lm(T2~(female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)

coeftest(m, vcov = vcovHC(m, type="HC1"))


t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
(Intercept)           0.5171342  0.1568823  3.2963 0.0009856 ***
female                0.0056240  0.1324597  0.0425 0.9661352    
black                -0.0307771  0.0847262 -0.3633 0.7164293    
othrace              -0.4278818  0.3005502 -1.4237 0.1545993    
factor(dep)1          0.1576567  0.1851798  0.8514 0.3945992    
factor(dep)2         -0.1927958  0.1508926 -1.2777 0.2014067    
q2                   -0.1092161  0.1583768 -0.6896 0.4904762    
q3                   -0.1075663  0.1579864 -0.6809 0.4959890    
q4                   -0.0621302  0.1581168 -0.3929 0.6943797    
q5                   -0.0498684  0.1575034 -0.3166 0.7515450    
q6                   -0.2176336  0.1558290 -1.3966 0.1625831    
agelt35              -0.0643464  0.1326376 -0.4851 0.6276032    
agegt54               0.0259251  0.2395371  0.1082 0.9138171    
durable              -0.1256397  0.1763853 -0.7123 0.4763064    

In [41]:
# get same dataframe

X <- as.data.frame( model.matrix(m) )

no_col <- attributes(m$coefficients)$names[is.na(m$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]
names(X1)[2] <- "T2"
save( X1, file = "../data/m_reg.RData")

### Generating similar qr decomposition

#### Replicate how lm drop variables [as discussed here](https://stackoverflow.com/questions/35410769/remove-perfectly-multicollinear-variables-from-data-frame)

We see that that even though this is a randomized experiment, balance conditions are failed.

# 1.1 Model Specification

In [42]:
# model specifications


# no adjustment (2-sample approach)
formula_cl <- log(inuidur1)~T2

# adding controls

formula_cra <- log(inuidur1)~T2+ (female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2

# Omitted dummies: q1, nondurable, muld


ols.cl_reg <- lm(formula_cl)
ols.cra_reg <- lm(formula_cra)


ols.cl = coeftest(ols.cl_reg, vcov = vcovHC(ols.cl_reg, type="HC1"))
ols.cra = coeftest(ols.cra_reg, vcov = vcovHC(ols.cra_reg, type="HC1"))

print(ols.cl)
print(ols.cra)




t test of coefficients:

             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.056830   0.020954 98.1580  < 2e-16 ***
T2TRUE      -0.078825   0.032509 -2.4247  0.01535 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
(Intercept)           2.4471832  0.3580899  6.8340 9.123e-12 ***
T2TRUE               -0.0712066  0.0320440 -2.2222 0.0263124 *  
female               -0.9428048  0.2827961 -3.3339 0.0008620 ***
black                -0.6654258  0.2430567 -2.7377 0.0062056 ** 
othrace              -1.5494550  0.7312071 -2.1190 0.0341306 *  
factor(dep)1          0.2798498  0.3783316  0.7397 0.4595160    
factor(dep)2          0.1678164  0.3678718  0.4562 0.6482768    
q2                    0.0271228  0.3606226  0.0752 0.9400495    
q3                   -0.3542370  0.3601780 -0.9835 0.3254005    
q4                   -0.3380244  0.3606994 -0.9371 0.3487284    
q5 

In [43]:
# interactive regression model variables 

X <- model.matrix(~(female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)[,-1] #without intercept

dim(X)

[1] 5782  119

In [44]:
# get same dataframe for ols.cl
#X <- as.data.frame( model.matrix(ols.cl_reg) )

#no_col <- attributes(ols.cl_reg$coefficients)$names[is.na(ols.cl_reg$coefficients)]

#X1  <- X[, -which(names( X ) %in% no_col ) ]

#save( X1, file = "../../../data/ols_cl_reg.RData")

# get same dataframe for ols.cl

X <- as.data.frame( model.matrix(ols.cra_reg) )

no_col <- attributes(ols.cra_reg$coefficients)$names[is.na(ols.cra_reg$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]


names(X1)[2] <- "T4"
save( X1, file = "../data/ols_cra_reg.RData")


In [45]:
X <- X[,-2]
head(X)

(Intercept) female black othrace factor(dep)1 factor(dep)2 q2 q3 q4 q5 ...
1 1           0      0     0       0            1            0  0  0  1  ...
2 1           0      0     0       0            0            0  1  0  0  ...
3 1           0      0     0       0            0            0  0  0  1  ...
4 1           0      0     0       0            0            0  0  1  0  ...
5 1           1      0     0       0            0            0  0  1  0  ...
6 1           0      0     0       1            0            0  0  0  1  ...
  agelt35:agegt54 agelt35:durable agelt35:lusd agelt35:husd agegt54:durable
1 0               0               0            0            0              
2 0               0               1            0            0              
3 0               0               0            0            0              
4 0               0               0            0            0              
5 0               0               0            0            0              
6 0               0               0            0            1              
  agegt54:lusd agegt54:husd durable:lusd durable:husd lusd:husd
1 0            0            0            0            0        
2 0            0            0            0            0        
3 0            0            0            0            0        
4 0            0            0            0            0        
5 0            0            0            0            0        
6 1            0            1            0            0

In [46]:
class(X1)

[1] "data.frame"

In [47]:
head(X1)

(Intercept) T4 female black othrace factor(dep)1 factor(dep)2 q2 q3 q4 ...
1 1           0  0      0     0       0            1            0  0  0  ...
2 1           1  0      0     0       0            0            0  1  0  ...
3 1           0  0      0     0       0            0            0  0  0  ...
4 1           0  0      0     0       0            0            0  0  1  ...
5 1           1  1      0     0       0            0            0  0  1  ...
6 1           0  0      0     0       1            0            0  0  0  ...
  q6:agegt54 q6:durable agelt35:durable agelt35:lusd agelt35:husd
1 0          0          0               0            0           
2 0          0          0               1            0           
3 0          0          0               0            0           
4 0          0          0               0            0           
5 0          0          0               0            0           
6 0          0          0               0            0           
  agegt54:durable agegt54:lusd agegt54:husd durable:lusd durable:husd
1 0               0            0            0            0           
2 0               0            0            0            0           
3 0               0            0            0            0           
4 0               0            0            0            0           
5 0               0            0            0            0           
6 1               1            0            1            0

The interactive specificaiton corresponds to the approach introduced in Lin (2013).

## 1.3 Interactive regression model

In [48]:

#interactive regression model

demean<- function(x){ x - mean(x)}
X = apply(X, 2, demean)

ols.ira_reg = lm(log(inuidur1) ~ T2*X) 
ols.ira= coeftest(ols.ira_reg, vcov = vcovHC(ols.ira_reg, type="HC1"))
print(ols.ira)






t test of coefficients:

                               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                   2.0556976  0.0207138 99.2430 < 2.2e-16 ***
T2TRUE                       -0.0699044  0.0320764 -2.1793 0.0293503 *  
Xfemale                      -0.6662823  0.4080234 -1.6330 0.1025357    
Xblack                       -0.8634862  0.2969435 -2.9079 0.0036528 ** 
Xothrace                     -3.8176881  0.9366193 -4.0760 4.645e-05 ***
Xfactor(dep)1                 0.0359264  0.6476816  0.0555 0.9557666    
Xfactor(dep)2                 0.2117556  0.4512231  0.4693 0.6388790    
Xq2                          -0.2546436  0.4553391 -0.5592 0.5760207    
Xq3                          -0.6212326  0.4549639 -1.3655 0.1721653    
Xq4                          -0.4799269  0.4561206 -1.0522 0.2927566    
Xq5                          -0.3718675  0.4538882 -0.8193 0.4126542    
Xq6                          -0.6770474  0.4521499 -1.4974 0.1343469    
Xagelt35                 

In [49]:
head(X)

(Intercept) female     black     othrace      factor(dep)1 factor(dep)2
1 0           -0.4007264 -0.118817 -0.005188516 -0.1165687    0.839329   
2 0           -0.4007264 -0.118817 -0.005188516 -0.1165687   -0.160671   
3 0           -0.4007264 -0.118817 -0.005188516 -0.1165687   -0.160671   
4 0           -0.4007264 -0.118817 -0.005188516 -0.1165687   -0.160671   
5 0            0.5992736 -0.118817 -0.005188516 -0.1165687   -0.160671   
6 0           -0.4007264 -0.118817 -0.005188516  0.8834313   -0.160671   
  q2         q3         q4         q5         ... agelt35:agegt54
1 -0.2001038 -0.2355586 -0.2232791  0.7367693 ... 0              
2 -0.2001038  0.7644414 -0.2232791 -0.2632307 ... 0              
3 -0.2001038 -0.2355586 -0.2232791  0.7367693 ... 0              
4 -0.2001038 -0.2355586  0.7767209 -0.2632307 ... 0              
5 -0.2001038 -0.2355586  0.7767209 -0.2632307 ... 0              
6 -0.2001038 -0.2355586 -0.2232791  0.7367693 ... 0              
  agelt35:durable agelt35:lusd agelt35:husd agegt54:durable agegt54:lusd
1 -0.07350398     -0.1324801   -0.1219301   -0.01885161     -0.0325147  
2 -0.07350398      0.8675199   -0.1219301   -0.01885161     -0.0325147  
3 -0.07350398     -0.1324801   -0.1219301   -0.01885161     -0.0325147  
4 -0.07350398     -0.1324801   -0.1219301   -0.01885161     -0.0325147  
5 -0.07350398     -0.1324801   -0.1219301   -0.01885161     -0.0325147  
6 -0.07350398     -0.1324801   -0.1219301    0.98114839      0.9674853  
  agegt54:husd durable:lusd durable:husd lusd:husd
1 -0.02767209  -0.05499827  -0.02905569  0        
2 -0.02767209  -0.05499827  -0.02905569  0        
3 -0.02767209  -0.05499827  -0.02905569  0        
4 -0.02767209  -0.05499827  -0.02905569  0        
5 -0.02767209  -0.05499827  -0.02905569  0        
6 -0.02767209   0.94500173  -0.02905569  0

In [50]:
# get same dataframe for ols.ira
S <- as.data.frame( model.matrix(ols.ira_reg) )

no_col <- attributes(ols.ira_reg$coefficients)$names[is.na(ols.ira_reg$coefficients)]

S1  <- S[, -which(names( S ) %in% no_col ) ]

names(S1)[2] <- "T2"
save( S1, file = "../data/ols_ira_reg.RData")



# 1.3 Next we try out partialling out with lasso

In [28]:
T2 = demean(T2)
DX = model.matrix(~T2*X)[,-1]
head(DX)

T2         X(Intercept) XT2TRUE    Xfemale    Xblack    Xothrace    
1 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
2  0.5800761 0             0.5800761 -0.4007264 -0.118817 -0.005188516
3 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
4 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
5  0.5800761 0             0.5800761  0.5992736 -0.118817 -0.005188516
6 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
  Xfactor(dep)1 Xfactor(dep)2 Xq2        Xq3        ... T2:Xagelt35:agegt54
1 -0.1165687     0.839329     -0.2001038 -0.2355586 ... 0                  
2 -0.1165687    -0.160671     -0.2001038  0.7644414 ... 0                  
3 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
4 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
5 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
6  0.8834313    -0.160671     -0.2001038 -0.2355586 ... 0                  
  T2:Xagelt35:durable T2:Xagelt35:lusd T2:Xagelt35:husd T2:Xagegt54:durable
1  0.03086608          0.05563156       0.05120138       0.007916241       
2 -0.04263790          0.50322755      -0.07072875      -0.010935367       
3  0.03086608          0.05563156       0.05120138       0.007916241       
4  0.03086608          0.05563156       0.05120138       0.007916241       
5 -0.04263790         -0.07684855      -0.07072875      -0.010935367       
6  0.03086608          0.05563156       0.05120138      -0.412007661       
  T2:Xagegt54:lusd T2:Xagegt54:husd T2:Xdurable:lusd T2:Xdurable:husd
1  0.0136537        0.01162017       0.02309509       0.01220118     
2 -0.0188610       -0.01605192      -0.03190318      -0.01685451     
3  0.0136537        0.01162017       0.02309509       0.01220118     
4  0.0136537        0.01162017       0.02309509       0.01220118     
5 -0.0188610       -0.01605192      -0.03190318      -0.01685451     
6 -0.4062702        0.01162017      -0.39682881       0.01220118     
  T2:Xlusd:husd
1 0            
2 0            
3 0            
4 0            
5 0            
6 0

In [29]:
library(hdm)

rlasso.ira = summary(rlassoEffects(DX, log(inuidur1), index = 1))

print(rlasso.ira)



Warning message:
"package 'hdm' was built under R version 4.0.5"


[1] "Estimates and significance testing of the effect of target variables"
    Estimate. Std. Error t value Pr(>|t|)  
T2 -6.431e+11  2.767e+11  -2.324   0.0201 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1




In [30]:
Lasso_baseline <- rlasso(x=DX,y=log(inuidur1))


In [31]:
Lasso_baseline$coefficients

(Intercept)                       T2             X(Intercept) 
               2.0237291                0.0000000                0.0000000 
                 XT2TRUE                  Xfemale                   Xblack 
               0.0000000                0.0000000               -0.3449936 
                Xothrace            Xfactor(dep)1            Xfactor(dep)2 
               0.0000000                0.0000000                0.0000000 
                     Xq2                      Xq3                      Xq4 
               0.0000000                0.0000000                0.0000000 
                     Xq5                      Xq6                 Xagelt35 
               0.0000000                0.0000000               -0.1588635 
                Xagegt54                 Xdurable                    Xlusd 
               0.1606780                0.0000000                0.0000000 
                   Xhusd            Xfemale:black          Xfemale:othrace 
               0.0000000                0.0000000                0.0000000 
    Xfemale:factor(dep)1     Xfemale:factor(dep)2               Xfemale:q2 
               0.0000000                0.0000000                0.0000000 
              Xfemale:q3               Xfemale:q4               Xfemale:q5 
               0.0000000                0.0000000                0.0000000 
              Xfemale:q6          Xfemale:agelt35          Xfemale:agegt54 
               0.0000000                0.0000000                0.0000000 
         Xfemale:durable             Xfemale:lusd             Xfemale:husd 
               0.0000000                0.0000000                0.0000000 
          Xblack:othrace      Xblack:factor(dep)1      Xblack:factor(dep)2 
               0.0000000                0.0000000                0.0000000 
               Xblack:q2                Xblack:q3                Xblack:q4 
               0.0000000                0.0000000                0.0000000 
               Xblack:q5                Xblack:q6           Xblack:agelt35 
               0.0000000                0.0000000                0.0000000 
          Xblack:agegt54           Xblack:durable              Xblack:lusd 
               0.0000000                0.0000000                0.0000000 
             Xblack:husd    Xothrace:factor(dep)1    Xothrace:factor(dep)2 
               0.0000000                0.0000000                0.0000000 
             Xothrace:q2              Xothrace:q3              Xothrace:q4 
               0.0000000                0.0000000                0.0000000 
             Xothrace:q5              Xothrace:q6         Xothrace:agelt35 
               0.0000000                0.0000000                0.0000000 
        Xothrace:agegt54         Xothrace:durable            Xothrace:lusd 
               0.0000000                0.0000000                0.0000000 
           Xothrace:husd         Xfactor(dep)1:q2         Xfactor(dep)2:q2 
               0.0000000                0.0000000                0.0000000 
        Xfactor(dep)1:q3         Xfactor(dep)2:q3         Xfactor(dep)1:q4 
               0.0000000                0.0000000                0.0000000 
        Xfactor(dep)2:q4         Xfactor(dep)1:q5         Xfactor(dep)2:q5 
               0.0000000                0.0000000                0.0000000 
        Xfactor(dep)1:q6         Xfactor(dep)2:q6    Xfactor(dep)1:agelt35 
               0.0000000                0.0000000                0.0000000 
   Xfactor(dep)2:agelt35    Xfactor(dep)1:agegt54    Xfactor(dep)2:agegt54 
               0.0000000                0.0000000                0.0000000 
   Xfactor(dep)1:durable    Xfactor(dep)2:durable       Xfactor(dep)1:lusd 
               0.0000000                0.0000000                0.0000000 
      Xfactor(dep)2:lusd       Xfactor(dep)1:husd       Xfactor(dep)2:husd 
               0.0000000                0.0000000                0.0000000 
                  Xq2:q3                   Xq2:q4            

In [33]:
# getting data
S = as.data.frame(DX)
save( S, file = "../data/rlasso_ira_reg.RData")

In [34]:
head(S)

T2         X(Intercept) XT2TRUE    Xfemale    Xblack    Xothrace    
1 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
2  0.5800761 0             0.5800761 -0.4007264 -0.118817 -0.005188516
3 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
4 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
5  0.5800761 0             0.5800761  0.5992736 -0.118817 -0.005188516
6 -0.4199239 0            -0.4199239 -0.4007264 -0.118817 -0.005188516
  Xfactor(dep)1 Xfactor(dep)2 Xq2        Xq3        ... T2:Xagelt35:agegt54
1 -0.1165687     0.839329     -0.2001038 -0.2355586 ... 0                  
2 -0.1165687    -0.160671     -0.2001038  0.7644414 ... 0                  
3 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
4 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
5 -0.1165687    -0.160671     -0.2001038 -0.2355586 ... 0                  
6  0.8834313    -0.160671     -0.2001038 -0.2355586 ... 0                  
  T2:Xagelt35:durable T2:Xagelt35:lusd T2:Xagelt35:husd T2:Xagegt54:durable
1  0.03086608          0.05563156       0.05120138       0.007916241       
2 -0.04263790          0.50322755      -0.07072875      -0.010935367       
3  0.03086608          0.05563156       0.05120138       0.007916241       
4  0.03086608          0.05563156       0.05120138       0.007916241       
5 -0.04263790         -0.07684855      -0.07072875      -0.010935367       
6  0.03086608          0.05563156       0.05120138      -0.412007661       
  T2:Xagegt54:lusd T2:Xagegt54:husd T2:Xdurable:lusd T2:Xdurable:husd
1  0.0136537        0.01162017       0.02309509       0.01220118     
2 -0.0188610       -0.01605192      -0.03190318      -0.01685451     
3  0.0136537        0.01162017       0.02309509       0.01220118     
4  0.0136537        0.01162017       0.02309509       0.01220118     
5 -0.0188610       -0.01605192      -0.03190318      -0.01685451     
6 -0.4062702        0.01162017      -0.39682881       0.01220118     
  T2:Xlusd:husd
1 0            
2 0            
3 0            
4 0            
5 0            
6 0

### Results

In [310]:
install.packages("xtable")
library(xtable)
table<- matrix(0, 2, 4)
table[1,1]<-  ols.cl[2,1]
table[1,2]<-  ols.cra[2,1]
table[1,3]<-  ols.ira[2,1]
table[1,4]<-  rlasso.ira[[1]][1]

table[2,1]<-  ols.cl[2,2]
table[2,2]<-  ols.cra[2,2]
table[2,3]<-  ols.ira[2,2]
table[2,4]<-  rlasso.ira[[1]][2]


colnames(table)<- c("CL","CRA","IRA", "IRA w Lasso")
rownames(table)<- c("estimate", "standard error")
tab<- xtable(table, digits=5)
tab

print(tab, type="latex", digits=5)

Warning message:
"package 'xtable' is in use and will not be installed"

CL          CRA         IRA         IRA w Lasso
estimate       -0.08545541 -0.07968012 -0.07550055 -0.07888608
standard error  0.03585569  0.03559092  0.03560489  0.03555130

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% Thu Apr 21 02:06:32 2022
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & CL & CRA & IRA & IRA w Lasso \\ 
  \hline
estimate & -0.08546 & -0.07968 & -0.07550 & -0.07889 \\ 
  standard error & 0.03586 & 0.03559 & 0.03560 & 0.03555 \\ 
   \hline
\end{tabular}
\end{table}


Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


